Import necessary packages: 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import textwrap

from itertools import product
from os import listdir
from scipy import stats
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from statannot import add_stat_annotation
from statsmodels.stats.multitest import multipletests

from functions_rw import calc_D0, calc_D1, calc_D2
from functions_rw import concat_df
from functions_rw import get_gmm_fitted, get_fcfp_gmm_rw
from functions_rw import get_nonzero_features
from functions_rw import get_trained_RF
from functions_rw import transform

Initialize a number of variables: 

In [2]:
np.random.seed(2703)
index_pair = 0
N_REP = 6
N_CELLS_REP = 48
N_CELLS_I = 100000
N_ITER = 100
N_MIX = 400
N_SAMPLES_LEAF = 5
N_TREES = 400
FEATURES = ['FL1-H','FL3-H','FSC-H','SSC-H'] 
PATH_DATA = 'QC_flowAI_CSV/'
FILENAMES = sorted(listdir(PATH_DATA))
TYPE = 'full'

Read-in metadata and annotate labels ('CD' = 1 vs 'HC' = 0): 

In [3]:
target = pd.read_csv('Metadata_DC.csv', index_col=0, header=0)
label_y = 'Health status'
le = LabelEncoder()
target.loc[:,label_y] = le.fit_transform(target.loc[:,label_y])
target.loc[:,label_y] = target.loc[:,label_y].replace({0:1, 1:0})
index_healthy = target.loc[target.loc[:,label_y] == 0].index
index_cd = target.loc[target.loc[:,label_y] == 1].index

Read in genus-table and calculate relative abundances: 

In [4]:
genus_table = pd.read_table('OTU_tables/GenusAbundance_DiseaseCohort_nature24460.txt', index_col=0, header=0)
genus_comp = genus_table.div(genus_table.sum(axis=1), axis=0)
genus_comp = genus_comp.loc[target.index,:]

Select a pair ('CD' and 'HC') randomly and create a training and test set: 

In [5]:
rand_arr = np.arange(0,1914,66)
arr = np.arange(0,66)
arr = np.random.choice(arr,29,False)
np.random.shuffle(arr)
selected_arr = rand_arr + arr
idx = selected_arr[index_pair]
list_comb = [x for x in product(np.arange(0,29),np.arange(29,95))]
index_test = target.index[[list_comb[idx][0],list_comb[idx][1]]]
target_train = target.drop(index_test)
target_test = target.loc[index_test,:]

In [6]:
df_fcm_train = concat_df(target_train, N_CELLS_REP, PATH_DATA, FILENAMES)
df_fcm_train_trans = transform(df_fcm_train, FEATURES)

Fit a Gaussian Mixture Model (GMM) to the data and extract a fingerprint per sample: 

In [7]:
GMM = get_gmm_fitted(df_fcm_train_trans, FEATURES, N_MIX, TYPE, False)
fcfp_train = get_fcfp_gmm_rw(target_train, N_REP, N_CELLS_I, N_MIX, FEATURES, GMM, True, PATH_DATA, FILENAMES)
fcfp_test = get_fcfp_gmm_rw(target_test, N_REP, N_CELLS_I, N_MIX, FEATURES, GMM, True, PATH_DATA, FILENAMES)

In [8]:
FCFP_FEATURES = get_nonzero_features(fcfp_train)

Create cross-validation object: 

In [9]:
cv_object = StratifiedKFold(n_splits=10, shuffle=True)

Train Random Forest classifier: 

In [10]:
RF = get_trained_RF(fcfp_train, target_train.loc[fcfp_train.index,label_y], FCFP_FEATURES, N_TREES, N_SAMPLES_LEAF, N_ITER, cv_object) 

Predict label of unseen fingerprints: 

In [11]:
preds_test = pd.DataFrame(RF.predict(fcfp_test.loc[:,FCFP_FEATURES]), index = fcfp_test.index)
print(preds_test)
print(target_test.loc[:,'Health status'])

            0
Individual   
DC01        1
DC78        0
Individual
DC01    1
DC78    0
Name: Health status, dtype: int64


Predict the probability of a sample having the label '1' (needed to determine the AUROC):

In [12]:
preds_proba_test = pd.DataFrame(RF.predict_proba(fcfp_test.loc[:,FCFP_FEATURES])[:,1], index = fcfp_test.index)
print(preds_test)
print(target_test.loc[:,'Health status'])

            0
Individual   
DC01        1
DC78        0
Individual
DC01    1
DC78    0
Name: Health status, dtype: int64


Repeat the same procedure based on Genus abundances. First, create a train and test set:  

In [13]:
genus_comp_train = genus_comp.drop(index_test)
genus_comp_test = genus_comp.loc[index_test,:]
GENERA = list(genus_comp_train.columns)

Train a Random Forest classifier: 

In [14]:
RF = get_trained_RF(genus_comp_train, target_train.loc[genus_comp_train.index,label_y], GENERA, N_TREES, N_SAMPLES_LEAF, N_ITER, cv_object) 

In [15]:
preds_test = pd.DataFrame(RF.predict(genus_comp_test.loc[:,GENERA]), index = genus_comp_test.index)
print(preds_test)
print(target_test.loc[:,'Health status'])

            0
Individual   
DC01        1
DC78        0
Individual
DC01    1
DC78    0
Name: Health status, dtype: int64


In [16]:
preds_proba_test = pd.DataFrame(RF.predict_proba(genus_comp_test.loc[:,GENERA])[:,1], index = genus_comp_test.index)
print(preds_test)
print(target_test.loc[:,'Health status'])

            0
Individual   
DC01        1
DC78        0
Individual
DC01    1
DC78    0
Name: Health status, dtype: int64
